Version 2

In [1]:

# --- Setup (no tensorflow_addons needed) ---
import os, random, math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tensorflow.keras import layers as L, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

SEED = 13
tf.random.set_seed(SEED); np.random.seed(SEED); random.seed(SEED)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 2
CLASS_NAMES = ['O', 'R']  # adjust to your folder names if needed


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("techsash/waste-classification-data")

print("Path to dataset files:", path)

100%|██████████| 427M/427M [00:03<00:00, 120MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/techsash/waste-classification-data/versions/1


In [3]:

# --- Paths (adjust if needed) ---
# Assuming the dataset is structured as: path/DATASET/TRAIN and path/DATASET/TEST
data_root = os.path.join(path, "DATASET") # Adjust this if the actual structure is different
train_dir = os.path.join(data_root, "TRAIN")
val_dir   = os.path.join(data_root, "TEST")   # Using TEST as validation/test split for convenience
test_dir  = val_dir

for p in [train_dir, val_dir]:
    if not os.path.isdir(p):
        raise FileNotFoundError(f"Directory not found: {p}")
print("Train:", train_dir, "\nVal/Test:", val_dir)

Train: /root/.cache/kagglehub/datasets/techsash/waste-classification-data/versions/1/DATASET/TRAIN 
Val/Test: /root/.cache/kagglehub/datasets/techsash/waste-classification-data/versions/1/DATASET/TEST


In [4]:

# --- Dataset loader using tf.data ---
def decode_img(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE, antialias=True)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)  # EfficientNet scaling
    return img, label

def build_paths_labels(root):
    paths, labels = [], []
    for idx, cname in enumerate(CLASS_NAMES):
        cdir = os.path.join(root, cname)
        if not os.path.isdir(cdir):
            cdir2 = os.path.join(root, cname.lower())
            cdir = cdir2 if os.path.isdir(cdir2) else cdir
        for r, _, files in os.walk(cdir):
            for f in files:
                if f.lower().endswith((".jpg", ".jpeg", ".png")):
                    paths.append(os.path.join(r, f))
                    labels.append(idx)
    return paths, labels

train_paths, train_labels = build_paths_labels(train_dir)
val_paths,   val_labels   = build_paths_labels(val_dir)
print(f"Train images: {len(train_paths)}  |  Val images: {len(val_paths)}")

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_ds   = tf.data.Dataset.from_tensor_slices((val_paths,   val_labels))

train_ds = (train_ds
            .shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
            .map(decode_img, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE)
            .prefetch(AUTOTUNE))

val_ds = (val_ds
          .map(decode_img, num_parallel_calls=AUTOTUNE)
          .batch(BATCH_SIZE)
          .prefetch(AUTOTUNE))


Train images: 22564  |  Val images: 2513


In [5]:

# --- Optional: class weights (helpful if imbalance) ---
from collections import Counter
cw = Counter(train_labels)
total = sum(cw.values())
class_weight = {i: total/(NUM_CLASSES*cw[i]) for i in range(NUM_CLASSES)}
print("Class counts:", dict(cw))
print("Class weights:", class_weight)


Class counts: {0: 12565, 1: 9999}
Class weights: {0: 0.8978909669717469, 1: 1.1283128312831283}


In [6]:

# --- Data augmentation block using Keras preprocessing layers ---
# These run on GPU and don't require addons.
data_augment = tf.keras.Sequential([
    L.RandomFlip("horizontal"),
    L.RandomRotation(0.08),   # ~±4.5 degrees
    L.RandomZoom(0.1),
    L.RandomContrast(0.1),
], name="augment")


In [ ]:

# --- Model: EfficientNetB0 + dropout ---
base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=IMG_SIZE+(3,))

inp = L.Input(shape=IMG_SIZE+(3,))
x = data_augment(inp)                      # augmentation inside the model
x = base(x, training=False)                # make sure BN runs in inference mode when frozen
x = L.GlobalAveragePooling2D()(x)
x = L.Dropout(0.25)(x)
out = L.Dense(1, activation="sigmoid")(x)
model = Model(inp, out)

# Phase 1: freeze backbone
for l in base.layers:
    l.trainable = False

loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
opt = Adam(learning_rate=1e-3)

model.compile(optimizer=opt, loss=loss,
              metrics=[
                  tf.keras.metrics.BinaryAccuracy(name="acc"),
                  tf.keras.metrics.AUC(name="auc")
              ])

ckpt = ModelCheckpoint("best_freeze.keras", monitor="val_auc", mode="max", save_best_only=True, verbose=1)
es   = EarlyStopping(monitor="val_auc", mode="max", patience=4, restore_best_weights=True)
rlr  = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1)

hist1 = model.fit(train_ds, validation_data=val_ds, epochs=8,
                  callbacks=[ckpt, es, rlr], class_weight=class_weight)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/8
706/706 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.9399 - auc: 0.7631 - loss: 0.2444
Epoch 1: val_auc improved from -inf to 0.98176, saving model to best_freeze.keras
706/706 ━━━━━━━━━━━━━━━━━━━━ 2079s 3s/step - acc: 0.9399 - auc: 0.7635 - loss: 0.2444 - val_acc: 0.9343 - val_auc: 0.9818 - val_loss: 0.2567 - learning_rate: 0.0010
Epoch 2/8
706/706 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.9516 - auc: 0.7907 - loss: 0.2149
Epoch 2: val_auc improved from 0.98176 to 0.98612, saving model to best_freeze.keras
706/706 ━━━━━━━━━━━━━━━━━━━━ 2082s 3s/step - acc: 0.9516 - auc: 0.7910 - loss: 0.2149 - val_acc: 0.9399 - val_auc: 0.9861 - val_loss: 0.2539 - learning_rate: 0.0010
Epoch 3/8
706/706 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.9611 - auc: 0.7929 - loss: 0.1997
Epoch 3: val_auc did not improve from 0.98612
706/706 ━━━━━━━━━━━━━━━━━━━━ 2088s 3s/step - acc: 0.9611 - auc: 0.7932 - loss: 0.1998 - val_acc: 0.9455 - val_auc: 0.9852 - val